# BMI 535/635: Management & Processing of Large-scale Data

#### Author: Michael Mooney (mooneymi@ohsu.edu)

## Data Storage and Querying Solutions in Python

1. Introduction
2. Learning Objectives
3. Resource Profiling
4. Review of Basic Python Data Types
5. Data from dbSNP
6. Connecting to Relational DBs
  - Object-relational Mapping (ORM)
7. Pandas
8. HDF5 (PyTables)
  - Hierarchical Data Format

Requirements:
- Python modules:
    - `os`
    - `time`
    - `timeit`
    - `memory_profiler`
    - `shutil`
    - `numpy`
    - `pandas`
    - `pytables (tables)`
    - `pymysql`
- Data files:
    - dbSNP annotations (chromosome 1 only): `chr1_reducedCols.txt.gz` (download this from the state server: /home/courses/BMI535/data/)
    - A MySQL config file containing connection parameters: `mysqlconfig.py`

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import tables
import pymysql as pym

## Introduction

Below are some common problems encountered when working with large datasets.

1. Data does not fit into memory
    - In particular, this can be a problem when setting up parallel computations, where each process needs the full data set
2. Accessing (querying) the data is slow
3. Data files on-disk are very large (i.e. not easily portable)

Potential Solutions:

1. Use on-disk storage that is optimized for fast read/write access
2. Use data storage that allows for multiple concurrent reads (i.e. can be shared across multiple processes)
3. Use data compression

### Learning Objectives

1. You will learn some basic methods for profiling the amount of resources and time used by computational tasks
2. You will learn how to store large datasets in various "high-performance" Python data structures
3. You will learn how to query data in each of the data structures
4. You will learn how to convert between these various data storage solutions


## Resource Profiling

More info on the `memory_profiler` module: [https://github.com/pythonprofilers/memory_profiler](https://github.com/pythonprofilers/memory_profiler)

In [2]:
## Note: this is not a python command (only needed in the Jupyter notebook)
%load_ext memory_profiler

In [3]:
import time
import timeit
from memory_profiler import memory_usage

In [4]:
help(memory_usage)

Help on function memory_usage in module memory_profiler:

memory_usage(proc=-1, interval=0.1, timeout=None, timestamps=False, include_children=False, multiprocess=False, max_usage=False, retval=False, stream=None, backend=None, max_iterations=None)
    Return the memory usage of a process or piece of code
    
    Parameters
    ----------
    proc : {int, string, tuple, subprocess.Popen}, optional
        The process to monitor. Can be given by an integer/string
        representing a PID, by a Popen object or by a tuple
        representing a Python function. The tuple contains three
        values (f, args, kw) and specifies to run the function
        f(*args, **kw).
        Set to -1 (default) for current process.
    
    interval : float, optional
        Interval at which measurements are collected.
    
    timeout : float, optional
        Maximum amount of time (in seconds) to wait before returning.
    
    max_usage : bool, optional
        Only return the maximum memory u

In [5]:
## A dummy function that creates a large list
def foo(a, n=100):
    time.sleep(2)
    b = [a] * n
    time.sleep(1)
    return None

## Use the time and memory_profiler modules to profile the foo function
t0 = time.time()
mem1 = memory_usage((foo, (1,10000000)), max_usage=True, timestamps=True)
print("Elapsed time: %.3f seconds\n Memory used: %.3f Mb" % (mem1[1]-t0, mem1[0]))

Elapsed time: 3.082 seconds
 Memory used: 180.488 Mb


In [6]:
## Use timeit to profile foo
timeit.timeit('foo(1,10000000)', setup='from __main__ import foo', number=1) 

3.0558501390000012

In [7]:
## Use timeit to profile multiple function calls
## Default is 5 repeats (repeat=5)
timeit.repeat('foo(1,10000000)', setup='from __main__ import foo', number=1) 

[3.054871234,
 3.053078629,
 3.055515139999999,
 3.056152695999998,
 3.0596082140000007]

In [8]:
help(timeit.repeat)

Help on function repeat in module timeit:

repeat(stmt='pass', setup='pass', timer=<built-in function perf_counter>, repeat=5, number=1000000, globals=None)
    Convenience function to create Timer object and call repeat method.



#### Note: in a Jupyter notebook the %memit, %time, and %timeit magics are available

Use the following to see the docstrings:

%memit?

%time?

%timeit?

In [9]:
%memit foo(1, 10000000)

peak memory: 180.64 MiB, increment: 0.00 MiB


In [10]:
%time foo(1, 10000000)

CPU times: user 46.4 ms, sys: 6.68 ms, total: 53.1 ms
Wall time: 3.06 s


In [11]:
%timeit -n 1 -r 3 foo(1, 10000000)

3.05 s ± 1.6 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


****Note: Be cautious when using these Jupyter magics when doing things such as opening files, it is possible your code will be executed multiple times which could cause problems (i.e. multiple open file handles).**

## Review of Basic Python Data Types

Basic Python data types and general rules for when to use them:

**Lists/Tuples**: Use these when you need to iterate over a collection of items.

**Dictionaries**: Use these when you need to repeatedly access individual data elements (e.g. searching by a key value). 

**Sets**: Use these when you need to test for membership in a collection of items. Note: dictionaries can work well for this as well.

In [12]:
%%memit
## Create some example data
import random
LIST1 = random.sample(range(1000000), 1000000)
DICT1 = dict([(i,idx) for idx, i in enumerate(LIST1)])
SET1 = set(LIST1)

peak memory: 404.97 MiB, increment: 224.33 MiB


In [13]:
%memit

peak memory: 404.95 MiB, increment: -0.03 MiB


In [14]:
## How long does it take to find an item?
## Using a list
t0 = time.time()
idx = LIST1.index(567890)
print(idx)
print("Elapsed time for list: %.3f seconds\n" % (time.time()-t0,))

## Using a dictionary
t0 = time.time()
idx = DICT1[567890]
print(idx)
print("Elapsed time for dictionary: %.3f seconds\n" % (time.time()-t0,))

884748
Elapsed time for list: 0.099 seconds

884748
Elapsed time for dictionary: 0.000 seconds



In [15]:
%time LIST1.index(567890)

CPU times: user 77.1 ms, sys: 648 µs, total: 77.8 ms
Wall time: 77.7 ms


884748

In [16]:
%time DICT1[567890]

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 4.77 µs


884748

In [17]:
## How long does it take to determine if an item exists?
x = 567890
## Using a list
print("List: ")
%time x in LIST1

## Using a dictionary
print("Dictionary: ")
%time x in DICT1

## Using a set
print("Set: ")
%time x in SET1

List: 
CPU times: user 111 ms, sys: 905 µs, total: 112 ms
Wall time: 112 ms
Dictionary: 
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 6.91 µs
Set: 
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs


True

## dbSNP Dataset

For the following examples, we'll be using data from dbSNP, which contains information about all single nucleotide polymorphisms (SNPs) on human chromosome 1. The data file is a tab-delimited text file containing four columns: the 'rs' number of the SNP, the chromosome, the position, and a comma-separated list of genes at the same location. Note: the file contains a multi-line header.

In [18]:
!head ./xdata/chr1_reducedCols.txt

dbSNP Chromosome Report
Refer to ftp://ftp.ncbi.nlm.nih.gov/snp/00readme for documentation on tabular data below

rs#	chr	chr	local
		pos	loci


171	1	175261679	
242	1	20869461	
538	1	6160958	KCNAB2


## Connecting to Relational DBs in Python

[https://pymysql.readthedocs.io/en/latest/](https://pymysql.readthedocs.io/en/latest/)

[https://github.com/PyMySQL/PyMySQL](https://github.com/PyMySQL/PyMySQL)

*For Postgres databases: [https://www.psycopg.org/psycopg3/docs/](https://www.psycopg.org/psycopg3/docs/). Usage is very similar.

The following MySQL examples assume a local database server, with a database called 'bmi535_snps'. The following commands were run to create a table and load data:

    CREATE TABLE snps (rs int(10), 
                       chr int(10), 
                       pos int(10), 
                       loci varchar(80));
    
    LOAD DATA LOCAL INFILE '~/Documents/github/large_scale_data/xdata/chr1_reducedCols.txt' 
    INTO TABLE snps FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n'
    IGNORE 7 LINES (rs, chr, pos, loci);
    
****Note: with newer versions of MySQL, you may need to change default settings to allow loading data from a local file. [https://dev.mysql.com/doc/refman/8.0/en/load-data-local.html](https://dev.mysql.com/doc/refman/8.0/en/load-data-local.html)


The following command was run to clean cases of missing data (un-mapped SNPs):

    UPDATE snps SET pos = NULL WHERE pos = 0;

I've also created a second table with the same data, but this time I've created an index on the 'pos' column.

    CREATE TABLE snps_idx SELECT * FROM snps;
    
    CREATE INDEX pos ON snps_idx (pos); 


****Note: The code below also requires that you create a python module named 'mysqlconfig' and save it in the current directory (or in your Python path). This module should simply define a dictionary named 'mysql' that includes entries for your host, database, user, and password settings.**


In [19]:
## Import database connection settings
import mysqlconfig as cfg
import base64

In [20]:
## Connect to the MySQL database
## Note: the 'cursorclass' parameter is optional, in this case it specifies
## that query results will be returned as dictionaries, rather than the default tuples
conn = pym.connect(host=cfg.mysql['host'], user=cfg.mysql['user'], password=base64.b64decode(cfg.mysql['password']), 
                   database=cfg.mysql['db'], cursorclass=pym.cursors.DictCursor)

In [21]:
%%time
query = "SELECT * FROM snps WHERE chr = 1 AND pos = 225512846 AND loci = 'DNAH14';"
with conn.cursor() as cur:
    cur.execute(query)
    result = cur.fetchone()
    print(result)

## if query is changing the DB, you need to explicitly commit the changes
#conn.commit()

{'rs': 189425743, 'chr': 1, 'pos': 225512846, 'loci': 'DNAH14'}
CPU times: user 1.07 ms, sys: 1.41 ms, total: 2.49 ms
Wall time: 5.89 s


In [22]:
%%time
## Now let's look at how an indexed table affects performance
query = "SELECT * FROM snps_idx WHERE chr = 1 AND pos = 225512846 AND loci = 'DNAH14';"
with conn.cursor() as cur:
    cur.execute(query)
    result = cur.fetchone()
    print(result)


{'rs': 189425743, 'chr': 1, 'pos': 225512846, 'loci': 'DNAH14'}
CPU times: user 895 µs, sys: 1.81 ms, total: 2.71 ms
Wall time: 2.86 ms


In [23]:
## Close the connection
conn.close()

In [24]:
## Note: you can use the following connection attribute to test if the connection is open
conn.open

False

## Object-relational Mapping (ORM)

ORM is a technique for translating data between a relational database (table structures) and data structures implemented in an object-oriented programming language. ORM methods address the challenges of "object-relational impedence mismatch". For example (from Ireland et al., 2009):

1. SQL does not allow for the specification of class hierarchies
2. How do we ensure state consistency between an object and a database row?
3. An object has an identity (memory location) apart from its state. 
4. A class definition is owned by a programming team, and a database schema is owned by a database team. How to we maintain consistency when changes are made to either?


A simple way to map a database row to an object, is to simply create a class with attributes for each column in the table:

    class Pet:
        name
        type

    class Person:
        first_name
        last_name

What about relationships between database tables. Pet owners might be represented in a few different ways:

    class Pet:
        name
        type
        owners ## list of Person objects

    class Person:
        first_name
        last_name
        pets ## list of Pet objects

    class Owner:
        Person
        Pet


A good example of the use of ORM in Python is the Django web framework, which allows for the development of database driven websites. 

[https://docs.djangoproject.com/en/2.1/topics/db/models/](https://docs.djangoproject.com/en/2.1/topics/db/models/)

[https://docs.djangoproject.com/en/2.1/topics/db/models/#relationships](https://docs.djangoproject.com/en/2.1/topics/db/models/#relationships)

## Pandas

Pandas is a Python package that defines specialized data structures and methods for data analysis. The Pandas dataframe was inspired by R dataframes. It is very similar to a numpy ndarray, but is extended to include indices. 

[https://pandas.pydata.org/pandas-docs/stable/index.html](https://pandas.pydata.org/pandas-docs/stable/index.html)

### Load Data into a Pandas DataFrame

In [25]:
## Load SNP Data into a Pandas DataFrame
## Note: we can load data directly from a compressed file (gzip)
snps = pd.read_csv('./xdata/chr1_reducedCols.txt.gz', compression='gzip', sep='\t', header=None, skiprows=7, names=['rs','chr','pos','loci'])

In [26]:
%memit

peak memory: 1803.82 MiB, increment: 0.00 MiB


In [27]:
## Check the dimensions of the dataframe
snps.shape

(12237943, 4)

In [28]:
## Print info about the data (data types, etc.)
snps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12237943 entries, 0 to 12237942
Data columns (total 4 columns):
rs      int64
chr     int64
pos     object
loci    object
dtypes: int64(2), object(2)
memory usage: 373.5+ MB


In [29]:
## Print the first few rows of the data
snps.head()

,rs,chr,pos,loci
0,171,1,175261679,NaN
1,242,1,20869461,NaN
2,538,1,6160958,KCNAB2
3,546,1,93617546,TMED5
4,549,1,15546825,TMEM51


In [30]:
## Do some data cleaning ...
## Since some SNP positions were missing (spaces), make sure to convert
## the column to numeric data.
## Also, fill NaNs in the loci column with empty strings. This will improve 
## compatability with other Python modules (e.g. conversion of data types)
snps['pos'] = pd.to_numeric(snps['pos'], errors='coerce', downcast='integer')
snps = snps.fillna({'loci':''})

In [31]:
snps.head()

,rs,chr,pos,loci
0,171,1,175261679.0,
1,242,1,20869461.0,
2,538,1,6160958.0,KCNAB2
3,546,1,93617546.0,TMED5
4,549,1,15546825.0,TMEM51


In [32]:
snps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12237943 entries, 0 to 12237942
Data columns (total 4 columns):
rs      int64
chr     int64
pos     float64
loci    object
dtypes: float64(1), int64(2), object(1)
memory usage: 373.5+ MB


### Perform Query Using Pandas

In [33]:
%time pandas_result = snps.query("(chr==1) & (pos==225512846) & (loci=='DNAH14')")

CPU times: user 946 ms, sys: 122 ms, total: 1.07 s
Wall time: 298 ms


In [34]:
pandas_result

,rs,chr,pos,loci
3456788,189425743,1,225512846.0,DNAH14


### Save to HDF5 for Use Later

**Pandas has some confusing documentation when it comes to creating HDF5 files from dataframes (the `to_hdf()` method). According to the docs, the `data_columns` parameter is meant to specify what columns should be indexed in the HDF5 file (PyTables format only). It does do this, but it also uses this parameter to specify which columns are able to be (easily) queried in the HDF5 table. And ultimately, whether or not indexes are created is controlled by the `index` parameter. As I see it, you should always use `data_columns=True` so you can always query all columns, but control indexing with the `index` parameter (and actually it is probably better to create indexes later, as needed, using the PyTables module; see below). Creating indexes on all columns is costly and probably unnecessary in most cases.

In [35]:
## Note: Use index=False to avoid creating any indexes in the HDF5 file.
%time snps.to_hdf('./xdata/snps_pandas_hdf.h5', mode='w', key='/snps', format='table', data_columns=True, index=False, complib='blosc:lz4', complevel=9)

CPU times: user 17.4 s, sys: 1.01 s, total: 18.4 s
Wall time: 10.1 s


In [36]:
## How much space is used on disk?
!du -sh ./xdata/snps_pandas_hdf.h5

160M	./xdata/snps_pandas_hdf.h5


In [37]:
## Save an HDF5 with zlib compression for compatibility with R
## This is much slower than above, so I've lowered the compression level
%time snps.to_hdf('./xdata/snps_pandas_hdf_zlib.h5', mode='w', key='/snps', format='table', data_columns=True, index=False, complib='zlib', complevel=6)

CPU times: user 1min 34s, sys: 1.79 s, total: 1min 36s
Wall time: 29.6 s


In [38]:
## How much space is used on disk?
!du -sh ./xdata/snps_pandas_hdf_zlib.h5

128M	./xdata/snps_pandas_hdf_zlib.h5


## Hierarchical Data Format (HDF)

HDF5 (the current version of HDF) is a file format, data model, and software library for working with HDF files. You can think of an HDF5 file as a container that can store and organized multiple heterogeneous datasets (much like a mini file system). The main components of an HDF5 file are:

- Groups
- Datasets
- Attributes (can be associated with both groups and datasets)

Every HDF5 file contains a root group. This root group can contain datasets and groups, which themselves can contain other groups or datasets. The following figure shows the structure of a file that contains two groups ('Viz' and 'SimOut') underneath the root group:<br />
<img src="./images/group.png" width="500" align="left" />

A dataset contains both metadata and the data itself:

<img src="./images/dataset.png" width="500" align="left" />
<br clear="all" />

Images from: [https://portal.hdfgroup.org/display/HDF5/Introduction+to+HDF5](https://portal.hdfgroup.org/display/HDF5/Introduction+to+HDF5)

## PyTables

PyTables is a Python module that provides an interface to the HDF5 library. It extends the basic HDF5 functionality to allow for improved querying.

[http://www.pytables.org/usersguide/tutorials.html](http://www.pytables.org/usersguide/tutorials.html)

Pytables allows for the storage of datasets containing both heterogeneous (Table objects, also known as compound datatypes in HDF) and homogeneous data (array objects). The main Python classes defined in PyTables, and their heirarchy, are as follows:

- Node
  - Group (representing an HDF5 group)
  - Leaf (representing datasets)
    - Table
    - CArray (compressible array)
    - EArray (enlargable array)
    - VLArray (variable-length array)

### HDF5 from Pandas (PyTables format)

In [39]:
## Set filename and determine whether the file is in PyTables format
pandas_hdf5 = './xdata/snps_pandas_hdf.h5'
tables.is_pytables_file(pandas_hdf5)

'2.1'

In [40]:
%%memit
## Let's load the HDF5 file exported by Pandas
h5file = tables.open_file(pandas_hdf5)
h5_snps_pandas = h5file.root.snps

peak memory: 2280.72 MiB, increment: -0.04 MiB


In [41]:
## Access the Table object under the snps group
h5_snps_pandas.table

/snps/table (Table(12237943,), shuffle, blosc:lz4(9)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "rs": Int64Col(shape=(), dflt=0, pos=1),
  "chr": Int64Col(shape=(), dflt=0, pos=2),
  "pos": Float64Col(shape=(), dflt=0.0, pos=3),
  "loci": StringCol(itemsize=76, shape=(), dflt=b'', pos=4)}
  byteorder := 'little'
  chunkshape := (4854,)

In [42]:
## Is the table indexed?
h5_snps_pandas.table.colindexes

{
  }

In [43]:
## Let's search the HDF5 file by iterating over all table rows
## The first column in the table is an index (row index from Pandas); I'm excluding it from the results
%time pytables_result = [row[1:] for row in h5_snps_pandas.table.iterrows() if row['chr']==1 and row['pos']==225512846 and row['loci']==b'DNAH14']

CPU times: user 3.33 s, sys: 101 ms, total: 3.43 s
Wall time: 3.47 s


In [44]:
pytables_result

[(189425743, 1, 225512846.0, b'DNAH14')]

In [45]:
## Now let's run a query using the PyTables in-kernel method
%time pytables_result2 = [row[1:] for row in h5_snps_pandas.table.where("""(chr==1) & (pos==225512846) & (loci=='DNAH14')""")]

CPU times: user 1.15 s, sys: 323 ms, total: 1.47 s
Wall time: 1.09 s


In [46]:
pytables_result2

[(189425743, 1, 225512846.0, b'DNAH14')]

In [47]:
## Close the file
h5file.close()

In [48]:
h5file.isopen

0

In [49]:
## Are any files still open?
len(tables.file._open_files.get_handlers_by_name('./xdata/snps_pandas_hdf.h5'))

2

In [50]:
## If so, close them
tables.file._open_files.close_all()

Closing remaining open files:./xdata/snps_pandas_hdf.h5...done./xdata/snps_pandas_hdf.h5...done


### PyTables: Creating an Index to Improve Query Performance

In [51]:
## Open the file in append mode
pandas_hdf5 = './xdata/snps_pandas_hdf.h5'
h5file = tables.open_file(pandas_hdf5, mode='a')
h5_snps_pandas = h5file.root.snps

In [52]:
## Let's take a look at the table
h5_snps_pandas.table

/snps/table (Table(12237943,), shuffle, blosc:lz4(9)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "rs": Int64Col(shape=(), dflt=0, pos=1),
  "chr": Int64Col(shape=(), dflt=0, pos=2),
  "pos": Float64Col(shape=(), dflt=0.0, pos=3),
  "loci": StringCol(itemsize=76, shape=(), dflt=b'', pos=4)}
  byteorder := 'little'
  chunkshape := (4854,)

In [53]:
## Now create the index
## Note: a csindex (completely sorted) is the most optimized index
## and therefore is likely to take longer to create and consume
## more disk space. You can create other types of indexes with
## the create_index() method
%time h5_snps_pandas.table.cols.pos.create_csindex()

CPU times: user 18.2 s, sys: 666 ms, total: 18.9 s
Wall time: 12.8 s


12237943

In [54]:
!du -sh ./xdata/snps_pandas_hdf.h5

208M	./xdata/snps_pandas_hdf.h5


In [55]:
## Now you should see that an index has been added
h5_snps_pandas.table

/snps/table (Table(12237943,), shuffle, blosc:lz4(9)) ''
  description := {
  "index": Int64Col(shape=(), dflt=0, pos=0),
  "rs": Int64Col(shape=(), dflt=0, pos=1),
  "chr": Int64Col(shape=(), dflt=0, pos=2),
  "pos": Float64Col(shape=(), dflt=0.0, pos=3),
  "loci": StringCol(itemsize=76, shape=(), dflt=b'', pos=4)}
  byteorder := 'little'
  chunkshape := (4854,)
  autoindex := True
  colindexes := {
    "pos": Index(9, full, shuffle, zlib(1)).is_csi=True}

In [56]:
## Determine whether your query will use an index
## This will return the column name whose index is usable, or
## an empty set if none
h5_snps_pandas.table.will_query_use_indexing("""(chr==1) & (pos==225512846) & (loci=='DNAH14')""")

frozenset({'pos'})

In [57]:
## Let's run a query using the PyTables in-kernel method (now using an index)
%time pandas_result2 = [row[1:] for row in h5_snps_pandas.table.where("""(chr==1) & (pos==225512846) & (loci=='DNAH14')""")]

CPU times: user 5.86 ms, sys: 7.8 ms, total: 13.7 ms
Wall time: 13.7 ms


****Note: this is comparable to using the indexed MySQL table!**

In [58]:
pandas_result2

[(189425743, 1, 225512846.0, b'DNAH14')]

In [59]:
## Close the file
h5file.close()

## How to create an HDF5 file from scratch (using PyTables)

Let's create an HDF5 file that contains a table and two carrays (2D matrices)...

***Important Note:** R stores matrices differently than Python. This results in matrices being transposed when created in Python then loaded in R. More info on why this occurs is here: https://cran.r-project.org/web/packages/reticulate/vignettes/arrays.html

In [60]:
## Open a new HDF5 file; Check if the file exists already (if so, delete)
new_file_path = './xdata/new_file.h5'
if os.path.exists(new_file_path):
    os.remove(new_file_path)

new_h5file = tables.open_file(new_file_path, 'w')

In [61]:
## Create a table definition
class SnpsTable(tables.IsDescription):
    rs = tables.IntCol(8, pos=0)
    chr = tables.IntCol(2, pos=1)
    pos = tables.FloatCol(pos=2)
    loci = tables.StringCol(76, pos=3)

## Create a group (to be consistent with the other files)
snps_grp = new_h5file.create_group(new_h5file.root, "snps", "SNPs")
    
## Create the empty table
tbl = new_h5file.create_table('/snps', 'snps_table', SnpsTable)

In [62]:
snps.head(10)

,rs,chr,pos,loci
0,171,1,175261679.0,
1,242,1,20869461.0,
2,538,1,6160958.0,KCNAB2
3,546,1,93617546.0,TMED5
4,549,1,15546825.0,TMEM51
5,568,1,203713133.0,ATP2B4
6,665,1,24181041.0,FUCA1
7,672,1,53679329.0,"C1orf123,CPT2"
8,677,1,173876561.0,SERPINC1
9,685,1,161191522.0,


In [63]:
## Append data to the table
row = tbl.row
for snp in snps.head(10).itertuples(index=False):
    row['rs'] = int(snp[0])
    row['chr'] = int(snp[1])
    row['pos'] = int(snp[2])
    row['loci'] = snp[3]
    row.append()

## Flush data in the table
tbl.flush()  
new_h5file.flush()

In [64]:
## Read from table
rows = tbl[0:5]
print(rows)

[(171, 1, 1.75261679e+08, b'') (242, 1, 2.08694610e+07, b'')
 (538, 1, 6.16095800e+06, b'KCNAB2') (546, 1, 9.36175460e+07, b'TMED5')
 (549, 1, 1.55468250e+07, b'TMEM51')]


In [65]:
## Create two carrays
## First create a new group in the file
matrices = new_h5file.create_group(new_h5file.root, "matrices", "Matrices")

In [66]:
## View attributes of the group
matrices._v_attrs

/matrices._v_attrs (AttributeSet), 3 attributes:
   [CLASS := 'GROUP',
    TITLE := 'Matrices',
    VERSION := '1.0']

In [67]:
## Add an attribute
matrices._v_attrs.dims1 = (100, 1000)
matrices._v_attrs.dims2 = (1000, 100)
matrices._v_attrs

/matrices._v_attrs (AttributeSet), 5 attributes:
   [CLASS := 'GROUP',
    TITLE := 'Matrices',
    VERSION := '1.0',
    dims1 := (100, 1000),
    dims2 := (1000, 100)]

In [68]:
## Now create the arrays
shape1 = (100, 1000)
shape2 = (1000, 100)
A = np.random.uniform(0, 1, size=shape1)
B = np.random.uniform(0, 1, size=shape2)
atom = tables.Atom.from_dtype(A.dtype)
filters = tables.Filters(complevel=6, complib='zlib')

new_h5file.create_carray(matrices, 'A', atom, shape1, 'Random Matrix A', filters=filters, obj=A)
new_h5file.create_carray(matrices, 'B', atom, shape2, 'Random Matrix B', filters=filters, obj=B)

new_h5file.flush()

In [69]:
new_h5file

File(filename=./xdata/new_file.h5, title='', mode='w', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/matrices (Group) 'Matrices'
/matrices/A (CArray(100, 1000), shuffle, zlib(6)) 'Random Matrix A'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (8, 1000)
/matrices/B (CArray(1000, 100), shuffle, zlib(6)) 'Random Matrix B'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (81, 100)
/snps (Group) 'SNPs'
/snps/snps_table (Table(10,)) ''
  description := {
  "rs": Int64Col(shape=(), dflt=0, pos=0),
  "chr": Int16Col(shape=(), dflt=0, pos=1),
  "pos": Float64Col(shape=(), dflt=0.0, pos=2),
  "loci": StringCol(itemsize=76, shape=(), dflt=b'', pos=3)}
  byteorder := 'little'
  chunkshape := (697,)

In [70]:
## Access the data
new_h5file.root.matrices.A[0:5,0:5]

array([[0.05107753, 0.88287307, 0.44200453, 0.87288683, 0.02865693],
       [0.61706907, 0.05712309, 0.45371441, 0.07868149, 0.7053214 ],
       [0.35338312, 0.23557121, 0.02670171, 0.09078162, 0.30163265],
       [0.14855962, 0.74098043, 0.74645748, 0.49955025, 0.89087437],
       [0.69476297, 0.65379209, 0.95070902, 0.34493422, 0.98218344]])

In [71]:
## Perform a dot product
%memit C = np.dot(new_h5file.root.matrices.A, new_h5file.root.matrices.B)

peak memory: 1957.06 MiB, increment: 1.66 MiB


In [72]:
C.shape

(100, 100)

In [73]:
## Close the file
new_h5file.close()

## What did we learn?

- Some basic ways to measure the performance (i.e. the resources used) of computational tasks
- How to connect to a DBMS in Python
- There are multiple solutions for storing large datasets in Python, each with different capabilities
- Indexing can greatly improve query performance

### A Quick Summary

- For storing data in memory:
    - Pandas (Numpy)
- For storing data on-disk:
    - HDF5 (PyTables)


## In-Class Exercises

In [74]:
## Exercise 1.
## What if you had matrices that didn't fit into memory? 
## Write an algorithm that performs a dot product by reading only 
## portions (blocks) of the matrix into memory.
##
## Create a new carray in the HDF5 file, called 'C', and write the 
## output of A . B to that array.



## References and Other Resources

- Ireland, C., Bowers, D., Newton, M., & Waugh, K. (2009). Understanding object-relational mapping: A framework based approach. International Journal on Advances in Software Volume 1, Numbers 2&3, 2009.
- Blaze: [http://blaze.pydata.org/](http://blaze.pydata.org/)
- Dask: [https://dask.pydata.org/en/latest/](https://dask.pydata.org/en/latest/)
- Bcolz:
[http://bcolz.readthedocs.io/en/latest/tutorial.html](http://bcolz.readthedocs.io/en/latest/tutorial.html)
    - Bcolz is a Python module for storing large data sets on-disk or in memory with compression. Bcolz stores data in a column-oriented manner, which can improve data access in certain cases. Column-oriented data structures can be beneficial when the workflow requires accessing a single column for all rows, as opposed to all data (all columns) for a single row. It also allows for efficiently adding/deleting columns in a dataset. 

#### Last Updated: 7-Jan-2022